### Abhängigkeiten Importieren

In [1]:
import ipycanvas
import ipywidgets
import numpy
import math

### Globale Konstanten

In [2]:
BOARD_SIZE = 8

NONE = 0
BLACK = 1
WHITE = 2

### Game State Klasse

In [3]:
class GameState:
    def __init__(self):
        self.board = numpy.zeros((8, 8), dtype=numpy.int8)
        self.board[3, 3] = WHITE
        self.board[3, 4] = BLACK
        self.board[4, 3] = BLACK
        self.board[4, 4] = WHITE
        self.frontier = {(2,2),(2,3),(2,4),(2,5),
                         (3,2),(3,5),(4,2),(4,5),
                         (5,2),(5,3),(5,4),(5,5)}
        self.turn = BLACK

In [38]:
directions = {(-1,-1),(0,-1),(1,-1),(-1,0),(1,0),(-1,1),(0,1),(1,1)}

In [35]:
class InvalidMoveException(Exception):
    pass

In [37]:
def opposite_player(player):
    return {BLACK: WHITE, WHITE: BLACK}[player]

### Makes move
<div class="alert alert-warning">
    <b>Warning:</b> Does not check if move is valid
</div>

In [39]:
def make_move(state, pos):
    for direction in directions:
        turn_in_direction
    state.board[pos] = state.turn
    state.turn = opposite_player(state.turn)

In [40]:
def turn_in_direction(state):

SyntaxError: unexpected EOF while parsing (<ipython-input-40-1994fbccac6e>, line 1)

In [27]:
def count_disks(state, player):
    return numpy.count_nonzero(state.board == player)

### Widgets Initialisieren

#### Canvas Widget Initialisieren

In [7]:
CELL_SIZE = 70

CANVAS_SIZE = BOARD_SIZE * CELL_SIZE

canvas = ipycanvas.MultiCanvas(2, width=CANVAS_SIZE, height=CANVAS_SIZE)
canvas[0].fill_style = 'darkgreen'
canvas[0].stroke_style = 'black'
canvas[0].fill_rect(0, 0, CANVAS_SIZE, CANVAS_SIZE)
canvas[0].begin_path()
for i in range(BOARD_SIZE+1):
    pos = i * CELL_SIZE
    canvas[0].move_to(pos, 0)
    canvas[0].line_to(pos, CANVAS_SIZE)
    canvas[0].move_to(0, pos)
    canvas[0].line_to(CANVAS_SIZE, pos)
canvas[0].stroke()

#### Score Widget initialisieren

In [21]:
score = ipywidgets.widgets.Label()

#### Output Widget initalisieren

In [9]:
output = ipywidgets.widgets.Output()

In [10]:
def display_board(state):
    update_output(state)
    display(canvas)
    display(score)
    display(output)

In [34]:
def update_output(state):
    with ipycanvas.hold_canvas(canvas):
        canvas[1].clear()
        for ((x, y), val) in numpy.ndenumerate(state.board):
            if val == NONE:
                continue
            elif val == BLACK:
                canvas[1].fill_style = 'black'
            else:
                canvas[1].fill_style = 'white'
            canvas[1].fill_arc((x + 0.5) * CELL_SIZE, (y + 0.5) * CELL_SIZE, CELL_SIZE / 2.2, 0, 2 * math.pi)
    score.value = f'Black Player : {count_disks(state, BLACK)} White Player : {count_disks(state, WHITE)}'

In [12]:
def mouse_down(x_px, y_px):
    with output:
        x = math.floor(x_px / CELL_SIZE)
        y = math.floor(y_px / CELL_SIZE)
        make_move(state, (x, y))
        update_output(state)

canvas[1].on_mouse_down(mouse_down)

In [13]:
state = GameState()

In [26]:
display_board(state)

MultiCanvas(height=560, width=560)

Label(value='Black Player : None     White Player : None')

Output()